# Benchmarks

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm
import pathlib

import numpy as np
import pandas as pd
import lifelines
import pandas as pd

from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import shutil

import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objects import Box

import matplotlib.pyplot as plt
from lifelines import CRCSplineFitter
import warnings
from lifelines.utils import CensoringType

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from IPython.display import clear_output
import pathlib

from lifelines.utils import concordance_index

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=1, threads_per_worker=10)
client = Client(cluster)

In [3]:
!hostname

s-sc-node002


In [4]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_name = "210714_metabolomics"
#data_path = "/data/analysis/ag-reils/steinfej"
data_pre = f"{base_path}/data/2_datasets_pre/{project_name}"
data_post = f"{base_path}/data/3_datasets_post/{project_name}"

project_label = "21_metabolomics_multitask"
project_path = f"{base_path}/results/projects/{project_label}"
figures_path = f"{project_path}/figures"
data_results_path = f"{project_path}/data"
pathlib.Path(figures_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(data_results_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [5]:
endpoints = [
    # Psychiatric/Neurological
    'M_all_cause_dementia',
]

In [6]:
partitions = [str(p) for p in range(0, 22)]
splits = ["train", "valid", "test"]

In [7]:
!ls {data_results_path}

MET_HRs_210819_metabolomics.feather
MET_HRs_210821_metabolomics.feather
NMR_acetone_IMPUTER.pkl
NMR_clinical_ldl_cholesterol_IMPUTER.pkl
REVISION_220110_attrib_raw.feather
REVISION_220111_temp_umap_shap.feather
add_data_211005.feather
adj_partial_effects_metabolomics_210914.feather
adj_partial_effects_metabolomics_210916.feather
adj_partial_effects_metabolomics_210920.feather
adj_partial_effects_metabolomics_210922A.feather
adj_partial_effects_metabolomics_211005_incl_cancer.feather
adj_partial_effects_metabolomics_211006_improved.feather
adj_partial_effects_metabolomics_211007.feather
attrib_raw_REVISION.feather
attrib_sample_wide_211028.feather
attributions_210915.feather
attributions_210916.feather
attributions_210917.feather
attributions_210924.feather
attributions_211008.feather
attributions_211014_Kernelexplainer.feather
attributions_211026.feather
benchmark1000_cindex_subgroups_220106.feather
benchmark100_cindex_subgroups_220103.feather
benchmark_cindex1000_centres_211007_220120

### Load models

In [17]:
import joblib
def get_cph(path): 
    with open(path,'rb') as f:
        cph = pickle.load(f)
    return cph#return pd.read_csv(f"{path[:-8]}.csv", index_col=0)

In [18]:
run = "220126_apoe4"

In [20]:
loghs = pd.read_feather(f"{data_results_path}/cox_{run}_metabolomics.feather").query("split=='test'")
cols = [col for col in loghs.columns if not "AgeSex" in col]
loghs = loghs[cols]

UndefinedVariableError: name 'split' is not defined

In [21]:
version=f"COX_{run}"
dump_path = f"{data_post}/{version}"
pathlib.Path(dump_path).mkdir(parents=True, exist_ok=True)

In [28]:
feature_sets = ["PANEL", "PANELapoe4"]

In [29]:
paths = [f"{dump_path}/DS_{endpoint}_{features}+Metabolomics_{partition}.p" 
         for features in feature_sets 
         for partition in partitions
        for endpoint in endpoints]

In [30]:
import glob, os
import glob
import pickle
import re
cph_dict = {}
for path in tqdm(paths):
    cph_dict[pathlib.Path(path).stem] = get_cph(path)

  0%|          | 0/44 [00:00<?, ?it/s]

In [32]:
endpoint="M_all_cause_dementia"
features = "PANEL"
partition = 2
cph = cph_dict[f"DS_{endpoint}_{features}+Metabolomics_{partition}"]

In [33]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 97676 total observations, 95644 right-censored observations>
             duration col = 'M_all_cause_dementia_event_time'
                event col = 'M_all_cause_dementia_event'
      baseline estimation = breslow
   number of observations = 97676
number of events observed = 2032
   partial log-likelihood = -22403.10
         time fit was run = 2022-01-26 14:26:27 UTC

---
                                             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                          
age_at_recruitment                           0.51       1.66       0.03             0.44             0.57                 1.56                 1.77
sex                                         -0.31       0.73       0.09            -0.48            -0.14                 0.62                 0.87
smoking_status_2.0                           0.15       1.16       0.07             0.01             0.29                 1.01                 1.33
alcohol_intake_frequency_2.0                -0.15       0.86       0.06            -0.27            -0.03                 0.77                 0.97
daily_physical_activity                     -0.02       0.98       0.02            -0.06             0.02                 0.94                 1.03
education_years                             -0.09       0.91       0.02            -0.13            -0.05                 0.88                 0.95
daily_healthy_food                          -0.11       0.90       0.17            -0.43             0.22                 0.65                 1.24
fh_diabetes                                 -0.02       0.98       0.06            -0.14             0.10                 0.87                 1.10
diabetes2                                    0.25       1.29       0.09             0.07             0.44                 1.07                 1.55
body_mass_index_bmi                         -0.16       0.85       0.18            -0.52             0.19                 0.60                 1.21
waist_hip_ratio                              0.03       1.03       0.06            -0.08             0.14                 0.92                 1.15
waist_circumference                          0.24       1.27       0.09             0.06             0.42                 1.06                 1.52
weight                                       0.00       1.00       0.21            -0.41             0.41                 0.66                 1.51
standing_height                             -0.18       0.84       0.12            -0.41             0.06                 0.66                 1.06
systolic_blood_pressure                     -0.03       0.97       0.02            -0.08             0.01                 0.92                 1.01
cholesterol                                  0.02       1.02       0.12            -0.22             0.26                 0.81                 1.30
ldl_direct                                  -0.02       0.98       0.11            -0.23             0.20                 0.79                 1.22
hdl_cholesterol                              0.01       1.01       0.05            -0.08             0.10                 0.92                 1.11
triglycerides                               -0.03       0.97       0.03            -0.09             0.03                 0.91                 1.03
glucose                                      0.03       1.03       0.02            -0.02             0.07                 0.98                 1.07
glycated_haemoglobin_hba1c                  -0.00       1.00       0.03            -0.05             0.05                 0.95                 1.05
creatinine                                  -0.06       0.94       0.04            -0.13             0.01                 0.88                 1.01
cystatin_c                        

In [34]:
endpoint="M_all_cause_dementia"
features = "PANELapoe4"
partition = 2
cph = cph_dict[f"DS_{endpoint}_{features}+Metabolomics_{partition}"]

In [35]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 97676 total observations, 95644 right-censored observations>
             duration col = 'M_all_cause_dementia_event_time'
                event col = 'M_all_cause_dementia_event'
      baseline estimation = breslow
   number of observations = 97676
number of events observed = 2032
   partial log-likelihood = -22285.21
         time fit was run = 2022-01-26 14:26:28 UTC

---
                                             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                          
age_at_recruitment                           0.51       1.67       0.03             0.45             0.58                 1.57                 1.78
sex                                         -0.31       0.73       0.09            -0.48            -0.14                 0.62                 0.87
smoking_status_2.0                           0.15       1.16       0.07             0.01             0.29                 1.01                 1.34
alcohol_intake_frequency_2.0                -0.16       0.86       0.06            -0.27            -0.04                 0.76                 0.96
daily_physical_activity                     -0.02       0.98       0.02            -0.07             0.02                 0.94                 1.02
education_years                             -0.09       0.92       0.02            -0.13            -0.05                 0.88                 0.96
daily_healthy_food                          -0.10       0.90       0.17            -0.42             0.22                 0.65                 1.25
fh_diabetes                                 -0.01       0.99       0.06            -0.13             0.11                 0.88                 1.12
diabetes2                                    0.23       1.26       0.09             0.04             0.41                 1.05                 1.51
body_mass_index_bmi                         -0.16       0.85       0.18            -0.52             0.20                 0.60                 1.22
waist_hip_ratio                              0.04       1.04       0.06            -0.07             0.15                 0.93                 1.16
waist_circumference                          0.23       1.26       0.09             0.06             0.41                 1.06                 1.51
weight                                       0.00       1.00       0.21            -0.41             0.41                 0.66                 1.51
standing_height                             -0.18       0.83       0.12            -0.42             0.05                 0.66                 1.06
systolic_blood_pressure                     -0.03       0.97       0.02            -0.08             0.01                 0.92                 1.01
cholesterol                                  0.01       1.01       0.12            -0.23             0.25                 0.79                 1.29
ldl_direct                                  -0.05       0.95       0.11            -0.26             0.17                 0.77                 1.18
hdl_cholesterol                              0.04       1.05       0.05            -0.05             0.14                 0.95                 1.15
triglycerides                               -0.02       0.98       0.03            -0.08             0.04                 0.92                 1.04
glucose                                      0.02       1.02       0.02            -0.02             0.07                 0.98                 1.07
glycated_haemoglobin_hba1c                   0.00       1.00       0.03            -0.05             0.05                 0.95                 1.06
creatinine                                  -0.06       0.94       0.03            -0.13             0.01                 0.88                 1.01
cystatin_c                        

In [37]:
from lifelines import utils
from lifelines.fitters import RegressionFitter, SemiParametricRegressionFitter
from lifelines.plotting import set_kwargs_drawstyle
def plot_partial_effects_on_outcome(cph, covariates, values, plot_baseline=True, y="survival_function", **kwargs):
    
        from matplotlib import pyplot as plt

        covariates = utils._to_list(covariates)
        n_covariates = len(covariates)
        values = np.asarray(values)
        if len(values.shape) == 1:
            values = values[None, :].T

        if n_covariates != values.shape[1]:
            raise ValueError("The number of covariates must equal to second dimension of the values array.")

        for covariate in covariates:
            if covariate not in cph._central_values.columns:
                raise KeyError("covariate `%s` is not present in the original dataset" % covariate)

        #drawstyle = "steps-post" if isinstance(cph._model, SemiParametricRegressionFitter) else None
        #set_kwargs_drawstyle(kwargs, drawstyle)
#
        if cph.strata is None:
            data_exp = []
            #axes = kwargs.pop("ax", None) or plt.figure().add_subplot(111)
            x_bar = cph._central_values
            X = pd.concat([x_bar] * values.shape[0])

            if np.array_equal(np.eye(n_covariates), values) or np.array_equal(
                np.append(np.eye(n_covariates), np.zeros((n_covariates, 1)), axis=1), values
            ):
                X.index = ["%s=1" % c for c in covariates]
            else:
                X.index = [", ".join("%s=%s" % (c, v) for (c, v) in zip(covariates, row)) for row in values]
            for covariate, value in zip(covariates, values.T):
                X[covariate] = value

            # if a column is typeA in the dataset, but the user gives us typeB, we want to cast it. This is
            # most relevant for categoricals.
            X = X.astype(cph._central_values.dtypes)

            #getattr(cph, "predict_%s" % y)(X).plot(ax=axes, **kwargs)
            data_exp = getattr(cph, "predict_%s" % y)(X)

        return data_exp

In [38]:
def get_part_effects_df(endpoint="M_MACE", features="Age+Sex", quantiles=[0.01, 0.1, 0.5, 0.9, 0.99], partition=0):
    cph = cph_dict[f'DS_{endpoint}_{features}+Metabolomics_{partition}']
    values = loghs[f'logh_{endpoint}_Metabolomics'].quantile(q=quantiles).to_list()
    data_exp = plot_partial_effects_on_outcome(cph,covariates=f'logh_{endpoint}_Metabolomics', values=values, cmap='coolwarm')
    return data_exp

def clean_df(df, quantiles):
    df.columns = quantiles+["endpoint", "features", "partition"]
    return df
    
quantiles=[0.01, 0.1, 0.5, 0.9, 0.99]
partial_effects = pd.concat([clean_df(get_part_effects_df(endpoint, features, quantiles, partition).assign(endpoint=endpoint, features=features, partition=partition), quantiles)
                             for endpoint in tqdm(endpoints)
                             for features in feature_sets
                             for partition in partitions
                            ])
partial_effects = partial_effects.reset_index().rename(columns={"index" : "time"})

  0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
partial_effects

,time,0.01,0.1,0.5,0.9,0.99,endpoint,features,partition
0,0.002738,0.999997,0.999996,0.999994,0.999992,0.999990,M_all_cause_dementia,PANEL,0
1,0.008214,0.999993,0.999991,0.999989,0.999984,0.999980,M_all_cause_dementia,PANEL,0
2,0.021903,0.999990,0.999987,0.999983,0.999976,0.999970,M_all_cause_dementia,PANEL,0
3,0.030116,0.999987,0.999982,0.999977,0.999968,0.999960,M_all_cause_dementia,PANEL,0
4,0.032854,0.999983,0.999978,0.999972,0.999960,0.999951,M_all_cause_dementia,PANEL,0
...,...,...,...,...,...,...,...,...,...
103513,13.911020,0.992583,0.990337,0.988079,0.983876,0.980331,M_all_cause_dementia,PANELapoe4,21
103514,13.913758,0.992583,0.990337,0.988079,0.983876,0.980331,M_all_cause_dementia,PANELapoe4,21
103515,13.916496,0.992583,0.990337,0.988079,0.983876,0.980331,M_all_cause_dementia,PANELapoe4,21
103516,13.924709,0.992583,0.990337,0.988079,0.983876,0.980331,M_all_cause_dementia,PANELapoe4,21


In [40]:
import plotly.express as px

df = partial_effects.melt(id_vars=["time", "endpoint", "features", "partition"], value_vars=quantiles, var_name="quantile", value_name="surv").assign(Ft=lambda x: 1-x.surv)
#fig = px.line(df, x="time", y="Ft", color='quantile', facet_col="features", template="ggplot2")
#fig.write_html("Figure4.html")

In [41]:
df.to_feather(f"{data_results_path}/adj_partial_effects_metabolomics_{run}.feather")

In [42]:
hrs = []
for key, cph in tqdm(cph_dict.items()):
    endpoint = cph.event_col[:-6]
    if "DS" in key:
        if "Metabolomics" in key:
            partition=int(key[-2:].replace("_", ""))
            hrs.append(cph.summary.assign(module="DS", endpoint=endpoint, features=key, partition=partition).reset_index())

  0%|          | 0/44 [00:00<?, ?it/s]

In [43]:
hrs_df = pd.concat(hrs, axis=0).reset_index(drop=True)

In [44]:
hrs_df.to_feather(f"{data_results_path}/hrs_metabolomics_{run}.feather")